# ЛАБОРАТОРНАЯ РАБОТА 3

## Проверка статистических гипотез. Гипотезы независимости

### Цель работы: 
Проверка статистических гипотез о однородности выборочных совокупностей в пакете STATISTICA, используя критерии: Хи-квадрат, t-критерий Стьюдента, непараметрический критерий Манна-Уитни.

### Задание 1

1.	Дана выборка ответов на вопросы социологического анкетирования 400 пенсионеров Томской области, представленная в виде числовых кодов ответов, соответствующих номеру ответа (файл “Данные”, лист “Ответы”). В соответствии с вариантом задания (см. ниже), установить существует ли зависимость между ответами респондентов на указанные в задании вопросы анкеты (файл “Анкета”), используя критерий независимости Хи-квадрат. Оценить степень и характер связи, используя выборочный коэффициент ранговой корреляции Спирмена.  

### вариант 2

In [23]:
# Импорт библиотек
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import scipy.stats as stats
import statsmodels.api as sm
from pylab import plt, mpl
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline
import numpy as np

In [228]:
df_task1 = pd.DataFrame({'№_вопроса' : ['в1'],
                        'признак': ['в13']})
df_task1

,№_вопроса,признак
0,в1,в13


###  1. С каким настроением Вы смотрите в будущее? (выберите один вариант)
1.	С надеждой и оптимизмом
2.	Спокойно, но без особых надежд
3.	С тревогой и неуверенностью
4.	Со страхом и отчаянием
5.	Затрудняюсь ответить



### 14 Обращались ли Вы и как часто за медицинской помощью за последние 12 месяцев? (вызов на дом, обращение в медицинское учреждение)
1.	Раз в неделю или чаще
2.	Один-два раза в месяц
3.	Один-два раза в полгода 
4.	Один раз за год
5.	Не обращались 




## Загрузка данных

In [229]:
data = pd.read_excel('LAB_DATA.xlsx', sheet_name=0)
data

,в1,в2_1,в2_2,в2_3,в2_4,в2_5,в2_6,в2_7,в2_8,в2_9,...,в71да_5,в71да_6,в71да_др,в72,в74,в75,в76,в78,в78_1,нас.пункт
0,1,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,2,4,4,1,1,Томск
1,3,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,2,4,4,1,1,Томск
2,1,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,2,3,1,1,1,Томск
3,3,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,...,NaN,1.0,нет жилья,1.0,3,4,1,1,1,Томск
4,3,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,2,3,1,1,1,Томск
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,3,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,3,4,1,3,8,Тегульдетский район
396,3,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,2,4,1,3,8,Тегульдетский район
397,2,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,2,4,4,3,8,Тегульдетский район
398,2,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,3,4,1,3,8,Тегульдетский район


### Создаем таблицу сопряжения 

In [230]:
df_crossx1 = pd.crosstab(data[data[df_task1.loc[0][0]] != 5][df_task1.loc[0][0]], 
                        data[data[df_task1.loc[0][1]] != 6][df_task1.loc[0][1]]).reset_index()

df_crossx1

в13,в1,1,2,3,4,5
0,1,2,27,70,18,1
1,2,0,22,105,25,7
2,3,1,6,50,30,8
3,4,0,1,9,5,2


In [231]:
df_crossx1.columns[1:]

Index([1, 2, 3, 4, 5], dtype='object', name='в13')

In [249]:
def chi2_test(df_task, df_crossx, datax ):
    '''
    df_task - выбор фильтрации ( по вариантам создан датафрейм)
    df_crossx - кростабулирвоанная таблица
    datax - датафрейм данных общий
    '''
    pearson_chi_square = stats.chi2_contingency(df_crossx[df_crossx.columns[1:]])[0]

    phi = np.sqrt((pearson_chi_square/np.matrix(df_crossx[df_crossx.columns[1:]]).sum()))

    contingency_coefficient = stats.contingency.association(df_crossx[df_crossx.columns[1:]], method='pearson')

    cramers = stats.contingency.association(df_crossx[df_crossx.columns[1:]], method='cramer')

    tau_b = stats.kendalltau(datax[df_task.loc[0][0]]
                         , datax[df_task.loc[0][1]], variant='b')[0]

    tau_c = stats.kendalltau(datax[df_task.loc[0][0]]
                         , datax[df_task.loc[0][1]], variant='c')[0]


    spearman_Rank_r = stats.spearmanr(datax[df_task.loc[0][0]]
                         , datax[df_task.loc[0][1]])[0]

    df = pd.DataFrame({'Statistic' :
                          ['Pearson Chi-square',
                          'Phi',
                          'Contingency coefficient',
                          'Cramer"V',
                          'Kendalls tau b',
                          'Kendals tau c',
                          'spearman_Rank_r'],
                          'Chi-square': [pearson_chi_square,
                                        phi,
                                        contingency_coefficient,
                                        cramers,
                                        tau_b,
                                        tau_c,
                                        spearman_Rank_r],
                          'P-value': [stats.chi2_contingency(df_crossx[df_crossx.columns[1:]])[1],
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      np.nan,
                                      stats.spearmanr(datax[df_task.loc[0][0]]
                         , datax[df_task.loc[0][1]])[1]]})

    return df    


In [247]:
df_task = df_task1
df_crossx =df_crossx1
datax = data 
chi2_test(df_task, df_crossx, data)

,Statistic,Chi-square,P-value
0,Pearson Chi-square,34.196098,6.281383e-04
1,Phi,0.296492,NaN
2,Contingency coefficient,0.284261,NaN
3,"Cramer""V",0.171180,NaN
4,Kendalls tau b,0.234900,NaN
5,Kendals tau c,0.188078,NaN
6,spearman_Rank_r,0.263834,8.538885e-08


Отметим, прежде всего, высоко значимое значение статистики Хи-квадрат:  . Это означает, что нулевая гипотеза должна быть отвергнута и признано значимым наличие связи между рассматриваемыми признаками. Связь между признаками есть.

### Задание 2

2.	2.	В файле “Данные” на листе “Котировки” приведены цены закрытия акций на ММВБ за период с 01.01.2015 по 01.09.2015 с периодичностью 1 день. Для указанных в варианте задания ценных бумаг, проверить гипотезу о независимости ежедневных относительных приращений котировок, используя критерий независимости Хи-квадрат. Оценить также степень и характер связи, используя выборочный коэффициент корреляции Пирсона.

In [235]:
df_task2 = pd.DataFrame({'Наим.Акции1' : ['MTSS'],
                         'Наим.Акции2' : ['SNGS']})
df_task2

,Наим.Акции1,Наим.Акции2
0,MTSS,SNGS


### Загрузка данных для задания 2

In [236]:
data2 = pd.read_excel('LAB_DATA.xlsx', sheet_name=1)
data2

,Дата,ROSN,AFLT,BANEP,VTBR,GAZP,GMNK,SIBN,LKON,MTSS,...,PLZL,RBSM,RUALR,SBER,SNGS,NVTK,TATN,RTKM,ODVA,RASP
0,20150105,196.75,33.21,935,0.06750,133.95,8590,142.0,2295.0,174.70,...,1000.5,3.625,385.7,56.37,24.240,455.5,238.00,91.44,0.1055,23.15
1,20150106,200.30,33.07,926,0.06666,138.92,9101,144.6,2345.0,179.90,...,1047.0,3.790,397.9,58.28,25.015,476.7,228.75,95.28,0.1195,23.43
2,20150108,214.95,35.17,956,0.06741,146.46,9550,148.1,2572.0,190.05,...,1060.0,3.875,403.6,65.70,26.265,483.0,245.00,90.10,0.1230,24.25
3,20150109,206.60,34.00,938,0.06531,141.70,9740,146.6,2461.0,195.00,...,1050.0,4.045,406.4,63.10,25.650,475.1,234.05,89.03,0.1195,23.80
4,20150112,204.00,34.45,935,0.06353,140.22,9839,143.1,2477.0,193.00,...,1020.0,3.885,411.9,62.90,25.450,470.7,228.25,88.80,0.1155,23.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,20150827,245.50,38.80,1324,0.06985,144.60,10371,145.5,2424.0,227.25,...,2280.0,4.950,300.0,73.80,34.150,599.5,306.00,79.29,0.1465,38.67
162,20150828,246.35,39.04,1357,0.06965,146.60,10430,147.6,2520.0,228.90,...,2370.0,4.960,302.5,74.82,34.825,613.9,313.00,81.45,0.1475,38.50
163,20150831,245.10,40.00,1359,0.06900,148.19,10569,146.9,2531.0,228.85,...,2370.0,4.930,300.5,74.50,34.295,633.0,323.00,81.49,0.1505,39.42
164,20150901,242.70,37.70,1356,0.06728,144.50,10329,146.5,2455.0,227.55,...,2465.0,4.940,296.0,73.14,33.750,610.0,310.00,79.62,0.1495,38.80


### Выборка относительных приращений

In [237]:
# Выборка значений
data_test = data2[[df_task2.loc[0][0], df_task2.loc[0][1]]]
# Выборка относительных приращений
sample_relatively = data_test.pct_change()
sample_relatively

,MTSS,SNGS
0,NaN,NaN
1,0.029765,0.031972
2,0.056420,0.049970
3,0.026046,-0.023415
4,-0.010256,-0.007797
...,...,...
161,0.028281,0.031722
162,0.007261,0.019766
163,-0.000218,-0.015219
164,-0.005681,-0.015892


In [238]:
# Удалим пропуски
df_fin1 =sample_relatively[sample_relatively.notna().any(axis=1)].copy()
df_fin1

,MTSS,SNGS
1,0.029765,0.031972
2,0.056420,0.049970
3,0.026046,-0.023415
4,-0.010256,-0.007797
5,0.059585,0.005894
...,...,...
161,0.028281,0.031722
162,0.007261,0.019766
163,-0.000218,-0.015219
164,-0.005681,-0.015892


In [239]:
df_fin1['bins1'] = pd.cut(df_fin1['SNGS'], bins=3)
cut_bins = [-0.0495, -0.0104, 0.0286, 0.0676]
df_fin1['bins2'] = pd.cut(df_fin1['MTSS'], bins=cut_bins)
df_fin1

,MTSS,SNGS,bins1,bins2
1,0.029765,0.031972,"(0.0286, 0.0676]","(0.0286, 0.0676]"
2,0.056420,0.049970,"(0.0286, 0.0676]","(0.0286, 0.0676]"
3,0.026046,-0.023415,"(-0.0495, -0.0104]","(-0.0104, 0.0286]"
4,-0.010256,-0.007797,"(-0.0104, 0.0286]","(-0.0104, 0.0286]"
5,0.059585,0.005894,"(-0.0104, 0.0286]","(0.0286, 0.0676]"
...,...,...,...,...
161,0.028281,0.031722,"(0.0286, 0.0676]","(-0.0104, 0.0286]"
162,0.007261,0.019766,"(-0.0104, 0.0286]","(-0.0104, 0.0286]"
163,-0.000218,-0.015219,"(-0.0495, -0.0104]","(-0.0104, 0.0286]"
164,-0.005681,-0.015892,"(-0.0495, -0.0104]","(-0.0104, 0.0286]"


In [240]:
df_fin2 = df_fin1.copy()
df_fin2[['bins1', 'bins2']] = df_fin2[['bins1', 'bins2']].astype('str')

In [241]:
df_crossx2 = pd.crosstab(df_fin2['bins1'], df_fin2['bins2']).reset_index()
df_crossx2.rename(columns={'(-0.0104, 0.0286]': 1, '(-0.0495, -0.0104]' : 2, '(0.0286, 0.0676]' : 3}, inplace=True)
df_crossx2

bins2,bins1,1,2,3
0,"(-0.0104, 0.0286]",69,11,10
1,"(-0.0495, -0.0104]",26,25,0
2,"(0.0286, 0.0676]",18,0,4


## Тест хи квадрат

In [248]:
df_task = df_task2
df_crossx =df_crossx2
datax = data2 
chi2_test(df_task, df_crossx, datax)

,Statistic,Chi-square,P-value
0,Pearson Chi-square,36.649606,2.126935e-07
1,Phi,0.474177,NaN
2,Contingency coefficient,0.428450,NaN
3,"Cramer""V",0.335294,NaN
4,Kendalls tau b,0.576496,NaN
5,Kendals tau c,0.576117,NaN
6,spearman_Rank_r,0.768572,1.240370e-33


## Хи квадрат

In [103]:
import numpy as np
cut_bins = [-0.03, -0.02, -0.01, 0,  0.02, 0.03, 0.05, 0.06, 0.1]
# Создаем переменную для AFLT
df_aflft = pd.DataFrame(df_fin1[df_task2.loc[0][0]])
df_aflft['bins1'] = pd.cut(df_aflft['AFLT'], bins = cut_bins)
aflft = df_aflft.groupby('bins1', as_index=False).agg('count')
aflft['group'] = np.arange(1,9)
aflft

,bins1,AFLT,group
0,"(-0.03, -0.02]",12,1
1,"(-0.02, -0.01]",22,2
2,"(-0.01, 0.0]",36,3
3,"(0.0, 0.02]",54,4
4,"(0.02, 0.03]",15,5
5,"(0.03, 0.05]",9,6
6,"(0.05, 0.06]",1,7
7,"(0.06, 0.1]",3,8


In [102]:
# Создаем переменную для AFLT
cut_bins = [-0.03, -0.02, -0.01, 0,  0.02, 0.03, 0.05, 0.06, 0.1]
df_SNGS = pd.DataFrame(df_fin1[df_task2.loc[0][1]])
df_SNGS['bins2'] = pd.cut(df_SNGS['SNGS'], bins=cut_bins)
SNGS = df_SNGS.groupby('bins2', as_index=False).agg('count')
SNGS['group'] = np.arange(1,9)
SNGS

,bins2,SNGS,group
0,"(-0.03, -0.02]",21,1
1,"(-0.02, -0.01]",21,2
2,"(-0.01, 0.0]",29,3
3,"(0.0, 0.02]",51,4
4,"(0.02, 0.03]",10,5
5,"(0.03, 0.05]",15,6
6,"(0.05, 0.06]",3,7
7,"(0.06, 0.1]",4,8


In [105]:
df_chi = SNGS[['group', 'SNGS']].merge(aflft[['group', 'AFLT']], how='left', on='group')
df_chi

,group,SNGS,AFLT
0,1,21,12
1,2,21,22
2,3,29,36
3,4,51,54
4,5,10,15
5,6,15,9
6,7,3,1
7,8,4,3


In [106]:
chi2_stat = stats.chi2_contingency(df_chi[['SNGS', 'AFLT']])
print(f'Статистика Chi-square = {chi2_stat[0]}')
print(f'p_value = {chi2_stat[1]}')

Статистика Chi-square = 6.947443740648707
p_value = 0.4343724677473162


Наблюдаемый уровень значимости   говорит о статистически не значимом различии выборочных распределений. Таким образом, нулевая гипотеза «Распределения доходностей акций  не различаются» принимается.

In [108]:
pvalue_ks = stats.ks_2samp(df_fin1['SNGS'], df_fin1['AFLT'])[1]

In [109]:
pvalue_ks

0.4208664797964598

In [128]:
def kolmogorov_tests(data1, data2):
    df = pd.DataFrame({'Mean Group 1' : [data1.mean()],
                      'Mean Group 2' : [data2.mean()],
                       'p-value': [stats.ks_2samp(data1, data2)[1]],
                      'Valid group1' : [len(data1)],
                      'Valid group2' : [len(data2)],
                        'Std group1' : [data1.std()],
                        'Std group2' :  [data2.std()]})
    return df

In [130]:
kolmogorov_tests(df_fin1['SNGS'], df_fin1['AFLT'])

,Mean Group 1,Mean Group 2,p-value,Valid group1,Valid group2,Std group1,Std group2
0,0.00227,0.000773,0.420866,165,165,0.024246,0.021967


### Вывод

Наблюдаемый уровень значимости   говорит о статистически не значимом различии выборочных распределений. Таким образом, нулевая гипотеза «Распределения доходностей акций  не различаются» принимается.